<a href="https://colab.research.google.com/github/sofiaif/detecciondefuego/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model

#cargamos los archivos que entrenamos para reutilizar nuestra red enuronal
longitud, altura = 150, 150
modelo = './modelo/modelo.h5'
pesos_modelo = './modelo/pesos.h5'
cnn = load_model(modelo)
cnn.load_weights(pesos_modelo)


#recibe el nomber de la imagen y nos dice clase de objeto que es
def predict(file):
  x = load_img(file, target_size=(longitud, altura))# a x le cargamosla imagen que queremos que nos prediga
  x = img_to_array(x)#convertimos la imagen en un arrglo
  x = np.expand_dims(x, axis=0)
  array = cnn.predict(x)
  result = array[0]
  answer = np.argmax(result)
  if answer == 0:
    print("pred: Perro")
  elif answer == 1:
    print("pred: Gato")
  elif answer == 2:
    print("pred: Gorila")

  return answer

OSError: ignored

In [ ]:
import sys #manejo de archivos 
import os #manejo de archivos 
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator #ayuda a preprocesar imagenes de nuestro al
from tensorflow.python.keras import optimizers #optimizador para entrenar nuesto algoritmo
from tensorflow.python.keras.models import Sequential #permite hacer redes neuronales secuenciales, c/ capa eat en orden
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D #permiten convolucion y maxpooling
from tensorflow.python.keras import backend as K #limpia sesion de keras corrienod en el background

K.clear_session()



data_entrenamiento = './data/entrenamiento'
data_validacion = './data/validacion'

"""
Parameters
"""
epocas=20 #n° de veecs que vamos a iterar en nuestro set de datos en el entrenamiento
longitud, altura = 150, 150 #tamaño de procesamiento de imagenes cambiando a 150pixeles
batch_size = 32 #numero de imagenes que se manda a procesar en cada paso
pasos = 1000 # de veces que se procesa la informacion en cada epoca (1epoca 1000pasos)
validation_steps = 300 #despues de cada epoca de corren 300 pasos con nuestro set de datos de validacion, para que aprenda nuestro algoritmo
filtrosConv1 = 32 #n°de filtros que va a tener una convolucion
filtrosConv2 = 64
tamano_filtro1 = (3, 3) #tamaño del filtro que usa la comvolucion
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2) #tamaño de filtro en nuestro maxpooling
clases = 3 #cantidad de clases de clasificacion de objetos(gatos, perros, etc)
lr = 0.0004 #learning read es que tan grande hace los ajustes pa acercarse a una solucion optima


##Preparamos nuestras imagenes
# se crea un generador que dice como preprocesar nuestra informacion, para despues transformarlas
entrenamiento_datagen = ImageDataGenerator( #importado en la linea 3
    rescale=1./255, #reescala imagenes en vez de 1 a 255, todos nuestro valores de pixeles esten de 0 a 1, para hacer mas eficiente nuestro entrenamiento
    shear_range=0.2, #genera nuestras imagenes pero las inclinar para que el algoritmo aprenda a detectar en otras posiciones
    zoom_range=0.2, #genera nuestras imagenes pero les hace zoom para que el algoritmo aprenda a detectar imagenenes incompletas o secciones de esta
    horizontal_flip=True) #toma una imagnes y la invierte para que aprenda a distinguir direccionalidad

test_datagen = ImageDataGenerator(rescale=1. / 255) #pasamos las imagenes tal cual son

"""entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento, #entra a nuestro directorio a la carpeta entrenamiento y abre todas las imagenes a
    target_size=(altura, longitud), # una altura y longitud especifica
    batch_size=batch_size, #procesarno en un batch ground de 32 ya definido arriba
    class_mode='categorical') #la clasificacion va a ser por categorias

validacion_generador = test_datagen.flow_from_directory( #igal para validacion
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')"""

entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
 #   class_mode='categorical')

validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
#    class_mode='categorical')



cnn = Sequential() #sera una red secuancial, es decir varias capas apiladas
cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding ="same", input_shape=(longitud, altura, 3), activation='relu')) #nuestra primera capa va a ser una convolucion,que va tener el numero de filtro definido, tamaño, padding le dice lo que hace nuetro filtro rn las esquinas, con imagenes a altura y longitud definida, funcion de activacion es relu
cnn.add(MaxPooling2D(pool_size=tamano_pool)) #luego se hace un maxpoolingo con un filtro de tamaño ya definido

cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding ="same")) #igual al filtro uno pero sin definir otros parametros que ya fueron filtrados en la capa anterior
cnn.add(MaxPooling2D(pool_size=tamano_pool))


"""INICIO DE CLASIFICACION"""
cnn.add(Flatten())#a las imagenes profundas la hacemos plana, en una imagen con toda la informacion de nuestra red neuronal
cnn.add(Dense(256, activation='relu'))#mandamos la imagen anterior a una capa normal, donde la capa estara conectada con todas las neuronas de la capa anterior
cnn.add(Dropout(0.5)) #durante el entrenamiento se le apaga un 50% de las neuronas cada paso para evitar sobre ajustar y no aprenda un caminoen especifico sino caminos alternos para recolectar data
cnn.add(Dense(clases, activation='softmax'))#ultima capa de 3 neuronas, softmax nos da un porcentaje de posibilidades de que sea de cada clase

cnn.compile(loss='categorical_crossentropy', #parametros para optimizar nuestro algoritmo:durante el entrenamiento ve la funcion de perdida, haciendo aproximaciones lr ya definidas, y la metrica es accuracy de que tan bien esta prendiendo
            optimizer=optimizers.Adam(lr=lr),
            metrics=['accuracy']) #porcentage de imagenes que esta clasificando bien



"""ENTRENAR NUESTRO ALGORITMO"""
cnn.fit_generator(
    entrenamiento_generador, #le pasamos nuestra imagenes de entrenamiento
    steps_per_epoch=pasos, #pasos por epocas
    epochs=epocas,
    validation_data=validacion_generador, #imegenes de validacion
    validation_steps=validation_steps) #pasos de validacion que ejecuta por cada epoca


"""GUARDAMOS NUESTRO MODELO EN UN ARCHIVO"""
target_dir = './modelo/'
if not os.path.exists(target_dir): # si no existe la carpeta la crea
  os.mkdir(target_dir)
cnn.save('./modelo/modelo.h5')
cnn.save_weights('./modelo/pesos.h5') #pesos de cada capas que ya entrenamos

SyntaxError: ignored